In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
from sklearn.model_selection import train_test_split

In [2]:
Xtrain_df = pd.read_csv(r"C:\Users\birud\OneDrive - iitgn.ac.in\github\Image Classifier\Step3 - Model Creation\Xtrain and Ytrain\Xtrain.csv")
Ytrain_df = pd.read_csv(r"C:\Users\birud\OneDrive - iitgn.ac.in\github\Image Classifier\Step3 - Model Creation\Xtrain and Ytrain\Ytrain.csv")

In [3]:
print(Xtrain_df.shape)
print(Ytrain_df.shape)

(622, 67500)
(622, 1)


In [4]:
new_dimensions = (150, 150)
# folder_name = 'Test_preprocessed'

def preprocessing(i):
        path = r'C:\Users\birud\OneDrive - iitgn.ac.in\github\Image Classifier\Step3 - Model Creation\Test_Images\%s.jpg' %(i)
        image = cv2.imread(path)
        # Convert to RGB model
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Resize the image
        resized_image = cv2.resize(rgb_image, new_dimensions)

        # Normalize pixel values to the range (0, 1)
        normalized_image = resized_image / 255.0
        pixels = np.array(normalized_image)
        flattened_pixels = pixels.reshape(-1)
        
        return flattened_pixels

In [5]:
train_data, test_data, train_labels, test_labels = train_test_split(Xtrain_df,Ytrain_df, test_size = 0.2, random_state = 42 )

In [6]:
model = Sequential(
    [               
        tf.keras.Input(shape=(67500,)), 
        Dense(units = 255, activation = "relu"),
        Dense(units = 150, activation = "relu"),
        Dense(units = 5, activation = "linear"),
    ], name = "image_classifier" 
)

In [7]:
model.summary()

Model: "image_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 255)               17212755  
                                                                 
 dense_1 (Dense)             (None, 150)               38400     
                                                                 
 dense_2 (Dense)             (None, 5)                 755       
                                                                 
Total params: 17251910 (65.81 MB)
Trainable params: 17251910 (65.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
[layer1, layer2, layer3] = model.layers
W1,b1 = layer1.get_weights()
W2,b2 = layer2.get_weights()
W3,b3 = layer3.get_weights()
print(f"W1 shape = {W1.shape}, b1 shape = {b1.shape}")
print(f"W2 shape = {W2.shape}, b2 shape = {b2.shape}")
print(f"W3 shape = {W3.shape}, b3 shape = {b3.shape}")

W1 shape = (67500, 255), b1 shape = (255,)
W2 shape = (255, 150), b2 shape = (150,)
W3 shape = (150, 5), b3 shape = (5,)


In [9]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

history = model.fit(
    Xtrain_df,Ytrain_df,
    epochs=20, batch_size = 32, validation_split = 0.2
)

Epoch 1/20


16/16 [==============================] - 2s 93ms/step - loss: 37.4987 - accuracy: 0.2656 - val_loss: 52.9841 - val_accuracy: 0.0000e+00
Epoch 2/20
16/16 [==============================] - 1s 81ms/step - loss: 11.0948 - accuracy: 0.3400 - val_loss: 48.6889 - val_accuracy: 0.0000e+00
Epoch 3/20
16/16 [==============================] - 1s 81ms/step - loss: 5.2636 - accuracy: 0.4346 - val_loss: 12.5616 - val_accuracy: 0.0320
Epoch 4/20
16/16 [==============================] - 1s 80ms/step - loss: 4.3493 - accuracy: 0.4366 - val_loss: 19.8285 - val_accuracy: 0.0000e+00
Epoch 5/20
16/16 [==============================] - 1s 81ms/step - loss: 2.6533 - accuracy: 0.5332 - val_loss: 18.0446 - val_accuracy: 0.0000e+00
Epoch 6/20
16/16 [==============================] - 1s 81ms/step - loss: 2.4965 - accuracy: 0.5392 - val_loss: 9.6653 - val_accuracy: 0.0480
Epoch 7/20
16/16 [==============================] - 1s 81ms/step - loss: 2.9862 - accuracy: 0.5091 - val_loss: 18.0152 - val_accu

In [10]:
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(test_accuracy)

4/4 [==============================] - 0s 8ms/step - loss: 4.1976 - accuracy: 0.7680
0.7680000066757202


In [35]:
# Load the test image
test_image = preprocessing(5)
print(test_image)

[0.83137255 0.83137255 0.86666667 ... 0.59215686 0.73333333 0.54509804]


In [36]:
prediction = model.predict(test_image.reshape(1,67500))  # prediction

print(f" predicting a Two: \n{prediction}")
print(f" Largest Prediction index: {np.argmax(prediction)}")

1/1 [==============================] - 0s 23ms/step
 predicting a Two: 
[[  9.825708    7.2254214   3.129969    1.9386996 -14.41153  ]]
 Largest Prediction index: 0


In [37]:
prediction_p = tf.nn.softmax(prediction)

print(f" predicting a Two. Probability vector: \n{prediction_p}")
print(f"Total of predictions: {np.sum(prediction_p):0.3f}")

 predicting a Two. Probability vector: 
[[9.2948544e-01 6.9016397e-02 1.1489993e-03 3.4910679e-04 2.7678589e-11]]
Total of predictions: 1.000


In [38]:
yhat = np.argmax(prediction_p)

print(f"np.argmax(prediction_p): {yhat}")

np.argmax(prediction_p): 0
